## Twilio sms Sample Code

* Sign up for a trial account at

>```https://www.twilio.com/try-twilio```

* Please install the twilio package in your conda environment by running the following command in the terminal

>``` pip install twilio ```

* Add the twilio API key parameters to your .env file

>```TWILIO_ACCOUNT_SID = "your-account-sid-goes-here"```
>```TWILIO_AUTH_TOKEN = "your-auth-token-goes-here"```

**REF: https://www.twilio.com/docs/libraries/python**
https://www.twilio.com/docs/sms/tutorials/how-to-send-sms-messages-python?code-sample=code-send-an-mms-message-with-an-image-url&code-language=Python&code-sdk-version=6.x

### Initialization

In [189]:
import os
from dotenv import load_dotenv
from twilio.rest import Client
import psycopg2
from datetime import date, timedelta
import numpy as np
import yfinance as yf

In [190]:
# Load .env environment variables
# Note: Replace below file name with your .env file name
load_dotenv("YH.env")

True

In [191]:
# Read the API keys from the .env file
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID") 
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")

# Create the api object
twilio_api = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

# Read connection string from the .env file 
conn_string = os.getenv("DB_STR") 

# This is a function that 
# calls the message stream object in twilio
def fetch_sms():
    return twilio_api.messages.stream()

In [192]:
# run the function above and capture the returned object
sms = fetch_sms()
sms

<generator object Version.stream at 0x7fc45224ff90>

In [193]:
# go through the list of messages 
for sms in twilio_api.messages.list():
    print(f"{sms.status} on {sms.date_sent} | direction {sms.direction} - message: {sms.body} to phone number {sms.from_}")

delivered on 2021-05-09 02:22:06+00:00 | direction outbound-api - message: Hi Rina, 
Welcome! 
 Please send us your tickers. Text TICKERS followed by your ticker list (e.g. TICKERS FB, AMD). to phone number +15046081628
delivered on 2021-05-09 02:22:05+00:00 | direction outbound-api - message: Hi Farah, 
Welcome! 
 Please send us your tickers. Text TICKERS followed by your ticker list (e.g. TICKERS FB, AMD). to phone number +15046081628
delivered on 2021-05-09 02:22:04+00:00 | direction outbound-api - message: Hi Farah, 
Welcome! 
 Please send us your tickers. Text TICKERS followed by your ticker list (e.g. TICKERS FB, AMD). to phone number +15046081628
delivered on 2021-05-09 02:22:03+00:00 | direction outbound-api - message: Hi Micheal, 
We have received your tickers! Your portfolio is being analyzed and we will send you the results shortly. to phone number +15046081628
delivered on 2021-05-09 02:22:02+00:00 | direction outbound-api - message: Hi Micheal, 
We have received your ticke

### Assumptions/Notes

In [194]:
# up to 10 tickers 
# command format 
# ADD Name
# TICKERS FB, ADM, AMD

### AUX Functions

In [195]:
def send_sms(name, phone, message):
    """Sends an sms message to a phone number

    Parameters
    ----------
    name: str
        The first name of the person
    phone: str
        The phone number to which the message is sent format +1xxxyyyzzzz
    message: str
        Body of text to be sent as a message

    Returns
    ----------
    None: This function does not return any value.

   """
    message = f"Hi {name}, \n" + message
    try:
        twilio_api.messages.create(
                     body=message,
                     from_='+15046081628',
                     to=f'{phone}'
        )
    except Exception as err:
        print(f"Error: '{err}'")

In [196]:
def parse_message(message_body):
    """Parses the given string by spliting it into 3 parts

    Parameters
    ----------
    message_body : str
        The message to be parsed

    Returns
    ----------
    str: Returns 2 strings.

   """

    message_split = message_body.split(' ', 1)
    
    if (len(message_split)<2) or message_split[1].replace(' ','')=='':
        message_split[0] = 'INVALID'
        message_split.append('')
    
    command = message_split[0].replace(' ', '')
    content = message_split[1].replace(' ', '')

    return command.strip(), content.strip()

In [197]:
def content_to_list(content):
    """Converts comma separated string to a list

    Parameters
    ----------
    content : str
        A comma separated string of ticker symbols.

    Returns
    ----------
    List: Returns the corresponding list item.
    
    """
    return content.replace(" ", "").split(',')

In [198]:
def is_ticker_valid(ticker):
    """Check whether given ticker is a valid stock symbol.

    Parameters
    ----------
    ticker : str
        A ticker symbol.

    Returns
    ----------
    bool: True if valid ticker, otherwise False.
    
    """
    try: 
        yf.Ticker(ticker).info['sector']
        return True
    except Exception as err:
        return False

In [199]:
def ticker_list_valid(ticker_list):
    for ticker in ticker_list:
        if is_ticker_valid(ticker)==False:
            print('Invalid ticker found. Halting!')
            return False
    return True

### DB related functions

In [200]:
def create_db_connection(conn_string):
    """Creates a connection to database

    Parameters
    ----------
    conn_string : str
        This is the connection string.

    Returns
    ----------
    obj: Returns a psycopg2.extensions.connection object.
    
    """
    conn = None
    try:
        conn = psycopg2.connect(conn_string)
        # for debug - to be removed for production
        print("MySQL Database connection successful")
    except Exception as err:
        print(f"Error: '{err}'")

    return conn

In [201]:
def execute_query(connection, query):
    """Executes a query to the databse.

    Parameters
    ----------
    connection : object 
        This is a psycopg2.extensions.connection object.
    query : str
        This is a SQL query string. 

    Returns
    ----------
    None: This function does not return any value.
    
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        # for debug - to be removed for production
        print("Query successful")
    except Exception as err:
        print(f"Error: '{err}'")

In [202]:
def select_from_db(connection, select_query):
    """Executes a query to the databse.

    Parameters
    ----------
    connection : object 
        This is a psycopg2.extensions.connection object.
    select_query : str
        This is a SQL select query string. 

    Returns
    ----------
    str: It returns a string value or an empty string if nothing found.
    
    """
    cursor = connection.cursor()
    try:
        cursor.execute(select_query)
        results = cursor.fetchone()
        connection.commit()
        # for debug - to be removed for production
        print("Select Query successful")
        if results != None:
            return results[0]
        else: return ""
    except Exception as err:
        print(f"Error: '{err}'")

In [203]:
def add_user(conn, name, number):
    """Adds a user to the databse 

    Parameters
    ----------
    conn : object 
        This is a psycopg2.extensions.connection object.
    name : str
        This is the name of the person.
    number : str
        This is the phone number of the person. 

    Returns
    ----------
    None: This function does not return any value.
    
    """
    #execute the sql 
    query = f"insert into client (user_name, phone_number) Select '{name}', '{number}' Where not exists(select * from client where phone_number='{number}')"
    execute_query(conn, query)
    conn.close()

In [204]:
def add_user_portfolio(conn, content, number):
    """Adds a portfolio to the database

    Parameters
    ----------
    conn : object 
        This is a psycopg2.extensions.connection object.
    content : str
        A comma separated string of ticker symbols.
    number : str
        This is the phone number of the person. 

    Returns
    ----------
    None: This function does not return any value.
    
    """
    query = f"insert into user_portfolio (user_id, tickers) values ((select user_id from client where phone_number='{number}'), '{content}') "
    execute_query(conn, query)
    conn.close()

In [205]:
def find_user_name(conn, number):
    """Finds the user's first name

    Parameters
    ----------
    conn : object 
        This is a psycopg2.extensions.connection object.
    number : str
        This is the phone number of the person. 

    Returns
    ----------
    str: The user's name.
    
    """
    #execute the sql 
    query = f"select user_name from client where phone_number='{number}'"
    result = select_from_db(conn, query)
    
    conn.close()
    return result

In [206]:
def execute_command(command, phone, content):
    """This functions executes commands received from the sms engine. 

    Parameters
    ----------
    command : str 
        This is the command type to be executed ADD = add user, TICKERS = add profile.
    phone : str
        This is the phone number of the person. 
    content : str
        This is the content to be added to the database.

    Returns
    ----------
    None: This function does not return any value.
    
    """
    welcome_message = 'Welcome! \n Please send us your tickers. Text TICKERS followed by your ticker list (e.g. TICKERS FB, AMD).'
    add_portfolio_message = 'We have received your tickers! Your portfolio is being analyzed and we will send you the results shortly.'
    
    if command=='ADD':
        # for debug - to be removed for production
        print('Adding a user!')
        # connect to db
        conn = create_db_connection(conn_string)
        # add user to db
        add_user(conn, content, phone)
        # send a welcome message to the user
        send_sms(content, phone, welcome_message)
        
    elif command=='TICKERS':
        # check if tickers are valid
        # get a list of user specified tickers
        ticker_list = content_to_list(content)
        if ticker_list_valid(ticker_list) == True:
            # for debug - to be removed for production
            print('Adding portfolio')
            # add portfolio to db
            conn = create_db_connection(conn_string)
            add_user_portfolio(conn, content, phone)
            # get the user name from db
            conn = create_db_connection(conn_string)
            name = find_user_name(conn, phone)
            # notify the user
            send_sms(name, phone, add_portfolio_message)
            # run MC sim - RN
            # add to events - RN
            # Visualize the data - JF
            # plots to be made  - JF
            # message back results - YH
        # message back with an error 
    else:  print('Invalid entry!')

### Main Program Area

In [207]:
# check inbound messages for today 

today = date.today()

for sms in twilio_api.messages.list():
    if (sms.direction == 'inbound') and (sms.date_created.date() == (today)):
        command, content = parse_message(sms.body)
        phone = sms.from_ #'+14162066136'
        execute_command(command.upper(), phone, content)
        print(sms.body)
        print('---')

Adding a user!
MySQL Database connection successful
Query successful
ADD 
Micheal
---
Invalid entry!
START
---
Invalid entry!
STOP
---
Adding portfolio
MySQL Database connection successful
Query successful
MySQL Database connection successful
Select Query successful
TICKERS TSLA, HD, GE
---
Adding portfolio
MySQL Database connection successful
Query successful
MySQL Database connection successful
Select Query successful
TICKERS TSLA
---
Invalid entry!
TICKER TSLA
---
Adding portfolio
MySQL Database connection successful
Query successful
MySQL Database connection successful
Select Query successful
Tickers TSLA
---
Adding a user!
MySQL Database connection successful
Query successful
ADD Farah 
---
Invalid entry!
TSLA, HD, GE
---
Adding a user!
MySQL Database connection successful
Query successful
ADD 
Farah 
---
Adding a user!
MySQL Database connection successful
Query successful
ADD Rina
---
Invalid entry!
2 +19175437108 TSLA, HD, GE
---
Invalid entry!
2 +15185888001 FB, AMZN
---
Invali